In [13]:
import pandas as pd
from pathlib import Path

# ─────────────────────────────────────────────
# 0. CSV 목록
# ─────────────────────────────────────────────
files = [
    "merged_data_m1_dong.csv",
    "merged_data_m1_yong.csv",
    "merged_data_m1_seoul.csv",
]

# ─────────────────────────────────────────────
# 1. 중복 헤더 고유화 함수 (pandas 버전 무관)
# ─────────────────────────────────────────────
def dedup_columns(cols):
    seen = {}
    new_cols = []
    for col in cols:
        col = str(col).strip()          # 공백 제거 & 문자열화
        if col == "" or col.lower() == "none":
            col = "EMPTY"               # 빈 헤더 임시 이름
        if col in seen:
            new_cols.append(f"{col}_{seen[col]}")  # col, col_1, col_2…
            seen[col] += 1
        else:
            new_cols.append(col)
            seen[col] = 1
    return new_cols

# ─────────────────────────────────────────────
# 2. CSV 읽고 정리
# ─────────────────────────────────────────────
dfs = []
for f in files:
    fp = Path(f)
    if not fp.exists():
        raise FileNotFoundError(fp)

    try:  # pandas 1.4+
        df = pd.read_csv(fp, encoding="utf-8-sig",
                         encoding_errors="replace", low_memory=False)
    except TypeError:  # pandas 1.3↓
        with open(fp, "r", encoding="utf-8-sig", errors="replace") as fh:
            df = pd.read_csv(fh, low_memory=False)

    # 헤더 클린 + 중복 처리
    df.columns = dedup_columns(df.columns)
    dfs.append(df)
    print(f"{fp.name:28} → {df.shape}")

# ─────────────────────────────────────────────
# 3. 헤더 동일성 검증
# ─────────────────────────────────────────────
if len({tuple(df.columns) for df in dfs}) != 1:
    raise ValueError("❌  CSV 헤더가 서로 다릅니다!")

# ─────────────────────────────────────────────
# 4. 병합 후 저장 (UTF-8 무 BOM)
# ─────────────────────────────────────────────
merged = pd.concat(dfs, ignore_index=True)
print("▶ 합친 DF shape :", merged.shape)

out_path = Path("merged_all_m1_clean.csv").resolve()
merged.to_csv(out_path, index=False, encoding="utf-8")
print("✅ 저장 완료 →", out_path)


merged_data_m1_dong.csv      → (862, 88)
merged_data_m1_yong.csv      → (3843, 88)
merged_data_m1_seoul.csv     → (3198, 88)
▶ 합친 DF shape : (7903, 88)
✅ 저장 완료 → /home/ccl/Desktop/SAFER/M1/webtest/merged_all_m1_clean.csv
